### Инструкция по выполнению

    1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
    2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!
    3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).
    4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.
    5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).
    6. Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?
    7. Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.421. При необходимости округляйте дробную часть до трех знаков.

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.



In [2]:
import pandas as pd
from pathlib import Path
path = Path.cwd()
path = path.joinpath('../data/raw/HSE_ML_week3')

data = pd.read_csv(path.joinpath('data-logistic.csv'), names=['target', 'attr1', 'attr2'])
data.head(10)

,target,attr1,attr2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750
5,1,1.454271,-0.665416
6,1,2.254227,2.263786
7,-1,-0.067580,1.469141
8,-1,-0.861961,-0.824856
9,1,0.699179,2.032488


In [38]:
from math import exp

def euclid_dist(x1, x2, y1, y2):
    return ((y1-x1)**2 + (y2-x2)**2)**0.5

def gradient_descent(w1, w2, C, k):
    new_w1 = w1 + k/data.shape[0]*data.apply(
        lambda row: row[0]*row[1]*(1 - 1/(1 + exp(-row[0]*(w1*row[1] + w2*row[2])))),
        axis=1
    ).sum() - k*C*w1
    new_w2 = w2 + k/data.shape[0]*data.apply(
        lambda row: row[0]*row[2]*(1 - 1/(1 + exp(-row[0]*(w1*row[1] + w2*row[2])))),
        axis=1
    ).sum() - k*C*w2
    
    return new_w1, new_w2
    

w1_prev = -1
w2_prev = -1
w1 = 0
w2 = 0
counter = 0
while euclid_dist(w1, w2, w1_prev, w2_prev) > 0.00001 and counter < 10000:
    w1_prev = w1
    w2_prev = w2
    w1, w2 = gradient_descent(w1, w2, C=0, k=1.5)
    
    counter += 1

print(counter, w1, w2)

21 0.2880984149587087 0.09171905740473663


**Вывод:** начальные w1, w2 влияют на число шагов, k сильно влияет на число шагов: например, при k = 1.5 сходимость получается за 26 шагов, а при k = 0.05 --- за 400 с чем-то.

In [32]:
from sklearn.metrics import roc_auc_score


y_true = data['target']
y_score = data.apply(
        lambda row: 1/(1 + exp(-w1*row[1] - w2*row[2])),
        axis=1
    )
roc_auc_score(y_true, y_score)

0.9268571428571428

In [44]:
w1_prev = -1
w2_prev = -1
w1 = 0
w2 = 0
counter = 0
while euclid_dist(w1, w2, w1_prev, w2_prev) > 0.00001 and counter < 10000:
    w1_prev = w1
    w2_prev = w2
    w1, w2 = gradient_descent(w1, w2, C=10, k=0.1)
    
    counter += 1

print(counter, w1, w2)

8 0.02855875454623421 0.024780137249735552


In [45]:
y_score = data.apply(
        lambda row: 1/(1 + exp(-w1*row[1] - w2*row[2])),
        axis=1
    )
roc_auc_score(y_true, y_score)

0.9362857142857142

In [46]:
file = open('/home/topcoder2k/HSE_ML/HSE_ML_week3_answers/logistic_regression.txt', 'w')
file.write('0.927 0.936')
file.close()